In [1]:
%%capture
!pip install torch
!pip install numpy --upgrade
!pip install pandas
!pip install scikit-learn

In [10]:
import torch
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

/Users/kedarthakkar/miniconda3/envs/book_embeddings_311/lib/python3.11/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [3]:
model = torch.load('../binaries/5k_steps.pt', map_location=torch.device('cpu'))

In [5]:
with open("../data/book_vectors.pickle", "rb") as handle:
    book_vectors = pickle.load(handle)

In [6]:
train_df = pd.read_csv("../data/train_data.csv")

In [7]:
book_to_index = {
    book_id: i for i, book_id in enumerate(train_df["book_id"].unique())
}

In [45]:
train_df.loc[train_df["title"].str.contains("lebron james", case=False)][["title", "book_id"]].drop_duplicates()

,title,book_id
1629987,The Whore of Akron: One Man's Search for the S...,10836035
2050589,King James: Believe the Hype---The LeBron Jame...,882588
2365034,"Return of the King: LeBron James, The Clevelan...",31742948


In [27]:
def compute_embedding(book_id):
    with torch.no_grad():
        return model.fc1(
            torch.tensor(book_vectors[book_to_index[book_id]].astype('float32').reshape(1, -1))
        )

In [28]:
# harry potter & another harry potter
cosine_similarity(compute_embedding(5), compute_embedding(3))

array([[0.9563586]], dtype=float32)

In [35]:
# harry potter & catch 22
cosine_similarity(compute_embedding(5), compute_embedding(255444))

array([[0.05244207]], dtype=float32)

In [42]:
# harry potter & lion witch wardrobe
cosine_similarity(compute_embedding(5), compute_embedding(100915))

array([[0.92751074]], dtype=float32)

In [44]:
# harry potter & the brothers grimm
cosine_similarity(compute_embedding(5), compute_embedding(13554713))

array([[0.27137864]], dtype=float32)

In [46]:
# harry potter & a lebron book
cosine_similarity(compute_embedding(5), compute_embedding(882588))

array([[-0.06296819]], dtype=float32)

In [55]:
%%time
for vector in list(book_vectors.values())[:10000]:
    with torch.no_grad():
        book_tensor = torch.tensor(vector.astype('float32').reshape(1, -1))
        book_embedding = model.fc1(book_tensor)

CPU times: user 1.1 s, sys: 117 ms, total: 1.22 s
Wall time: 730 ms
